# LCA with only background

### Preparation

1. Direct to the correct Brightway project and ensure the background database is loaded.

In [1]:
import bw2data as bd

bd.projects.set_current("advlca25")
bd.databases

Databases dictionary with 4 object(s):
	ALIGNED-biob-prod-dummy
	Pyro Phenol
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential

2. Importing required libraries

In [2]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bw_bamboo.background_importer import *
from bw_bamboo.foreground_importer import *
from bw_bamboo.datapackage_builder import *
from bw_bamboo.uncertainty_handler import *
from bw_bamboo.lca_wrapper import *
from bw_bamboo.uncertainty_importer import *

3. Define some constants

In [3]:
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - fossil - air",
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

METHOD = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')

# BACKGROUND DATABASE FILE PATH
EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/S.txt")

# FOREGROUND DATABASE FILE PATH
FROEGROUND_1 = os.path.join(os.getcwd(), "data/foreground_system_1.csv")
FROEGROUND_2 = os.path.join(os.getcwd(), "data/foreground_system_2.csv")

# CHARACTERIZATION FACTOR MAPPING FILE PATH
CF_MAPPING_FILE = os.path.join(os.getcwd(), "data/cf_mapping_file.csv")

### Step 1: Get the background database matrices

1. Get technosphere matrix

In [4]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.build_tech_matrix(raw_tech)

print(f"The shape of technosphere matrix is: {tech_matrix.shape}")

The shape of technosphere matrix is: (76, 76)


2. Get biosphere matrix

In [5]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.build_bio_matrix(bio_df, GHG)

print(f"The shape of biosphere matrix is: {bio_matrix.shape}")

The shape of biosphere matrix is: (18, 76)


3. Get characterization factor matrix  
You can choose to search for CFs from the biosphere database, or you can put CFs directly into the file.

In [6]:
# get characterization factor matrix from code.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG, "ecoinvent-3.11-biosphere", METHOD, source="code")

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 2.850000e+01
 2.970000e+01 2.970000e+01 2.970000e+01 2.970000e+01 2.970000e+01
 2.970000e+01 2.970000e+01 2.970000e+01 2.970000e+01 2.970000e+01
 2.648000e+02 2.648000e+02 2.350682e+04]


In [ ]:
# get characterization factor matrix from the file directly.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG)  # By default, source="cf", so you can omit some parameters.

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

### Step 2: Buiild datapackage

4. Build datapackage

In [7]:
dp_builder = DatapackageBuilder()
datapackage_data = dp_builder.prepare_dp_matrix(tech_matrix, bio_matrix, cf_matrix)
datapackage = dp_builder.prepare_datapackage(datapackage_data)

### Step 3: Run LCA

5. Run static simulation through brightway directly

In [8]:
import bw2calc as bc

mysector = 68
activity_name = "RoW-Services"

lca = bc.LCA(
    demand={mysector: 1},
    data_objs=[datapackage],
)
lca.lci()
lca.lcia()

print(f"{activity_name}: {lca.score}")

RoW-Services: 291733.4492960721


### Step 6: Compare to manually LCA

6. Compare with lca manually.

In [9]:
lca_wrapper = LCAWrapper()

manual_tech_Data = -tech_matrix
np.fill_diagonal(manual_tech_Data, -manual_tech_Data.diagonal())

manual_lca = lca_wrapper.manual_lca(manual_tech_Data, bio_matrix, cf_matrix, mysector)

print(f"{activity_name}: {manual_lca}")


RoW-Services: 291733.449296072
